In [1]:
import json
import os
from jira import JIRA
from dotenv import load_dotenv
from typing import List, Dict, Optional

class JiraClient:

    def __init__(self, server: str, username: str, password: str):
        self.server = server
        self.username = username
        self.password = password
        self.jira = self._connect()

    def _connect(self) -> JIRA:
        try:
            jira_options = {'server': self.server}
            return JIRA(options=jira_options, basic_auth=(self.username, self.password))
        except Exception as e:
            raise Exception(f"Failed to connect to Jira: {e}")

    def get_issue(self, issue_key: str) -> Optional[JIRA]:
        try:
            return self.jira.issue(issue_key)
        except Exception as e:
            print(f"Error getting issue {issue_key}: {e}")
            return None

    def get_card_data(self, card_key: str) -> Optional[Dict]:
        card_data = self.get_issue(card_key)
        if not card_data:
            return None

        return {
            "id": card_data.key,
            "title": card_data.fields.summary,
            "description": card_data.fields.description,
            "status": card_data.fields.status.name,
            "assignee": card_data.fields.assignee.displayName if card_data.fields.assignee else 'Unassigned'
        }


    def get_epic_data(self, epic_key: str) -> Optional[Dict]:
        epic_data = self.get_issue(epic_key)
        if not epic_data:
            return None

        return {
            "id": epic_data.key,
            "title": epic_data.fields.summary,
            "description": epic_data.fields.description,
            "status": epic_data.fields.status.name,
        }


    def get_issues_linked_to_epic(self, epic_key: str) -> Optional[List[str]]:
        try:
            jql = f'parent = {epic_key}'
            issues = self.jira.search_issues(jql, fields='key')
            return [issue.key for issue in issues]
        except Exception as e:
            print(f"Error getting linked issues for epic {epic_key}: {e}")
            return None

def main():

    load_dotenv()

    # Access the environment variables
    jira_server = os.getenv("JIRA_SERVER")
    jira_username = os.getenv("JIRA_USERNAME")
    jira_password = os.getenv("JIRA_PASSWORD")
    epic_key = os.getenv("EPIC_KEY")

    if not all([jira_server, jira_username, jira_password, epic_key]):
        print("Error: Missing required environment variables.")
        return

    jira_client = JiraClient(jira_server, jira_username, jira_password)

    # Get epic data
    epic_data = jira_client.get_epic_data(epic_key)
    if not epic_data:
        print(f"Failed to retrieve epic data for key: {epic_key}")
        return

    final_data = epic_data
    final_data["cards"] = []

    # Get cards linked to the epic
    linked_issue_keys = jira_client.get_issues_linked_to_epic(epic_key)
    if linked_issue_keys:
        for card_key in linked_issue_keys:
            card_data = jira_client.get_card_data(card_key)
            if card_data:
                final_data["cards"].append(card_data)
    print(final_data)
    new_filename = f"{epic_key}.json"

    # Write the data to a JSON file
    try:
        with open(new_filename, 'w') as f:
            json.dump(final_data, f, indent=4)
        print(f"Data successfully written to {new_filename}")
    except Exception as e:
        print(f"Error writing to JSON file: {e}")

if __name__ == "__main__":
    main()


/Users/riya.mathew/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'id': 'AD-10', 'title': 'Product Search Feature', 'description': None, 'status': 'To Do', 'cards': [{'id': 'AD-13', 'title': '[BE] Add get product endpoint', 'description': 'Add endpoint to fetch product by ID from product list', 'status': 'To Do', 'assignee': 'Tarushi Bhandari'}, {'id': 'AD-12', 'title': '[BFF] Add endpoint for fetching product details', 'description': 'Add endpoint to fetch product details using product id', 'status': 'To Do', 'assignee': 'Tarushi Bhandari'}, {'id': 'AD-11', 'title': '[FE] Searching Products', 'description': 'h1. Allow users to view products by searching their respective IDs.', 'status': 'To Do', 'assignee': 'Riya Sara Mathew'}]}
Data successfully written to AD-10.json
